In [2]:
import cv2
import os
from datetime import date
from datetime import datetime
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import joblib

In [3]:
face_detector = cv2.CascadeClassifier('model/haarcascade_frontalface_default.xml')
smile_detector = cv2.CascadeClassifier('model/haarcascade_smile.xml')

In [4]:
cap = cv2.VideoCapture(0)

In [5]:
datetoday = date.today().strftime("%m_%d_%y")
datetoday2 = date.today().strftime("%d-%B-%Y")

In [6]:
if not os.path.isdir('Attendance'):
    os.makedirs('Attendance')
if not os.path.isdir('static/faces'):
    os.makedirs('static/faces')
if f'Attendance-{datetoday}.csv' not in os.listdir('Attendance'):
    with open(f'Attendance/Attendance-{datetoday}.csv','w') as f:
        f.write('Name,Roll,Time')


In [7]:
def totalreg():
    return len(os.listdir('static/faces'))

In [8]:
def extract_faces(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    face_points = face_detector.detectMultiScale(gray, 1.3, 5)
    return face_points

In [9]:
def smile_detector(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    smile_points = smile_detector.detectMultiScale(gray, 1.7, 35)
    return smile_points

In [10]:
def identify_face(facearray):
    model = joblib.load('model/face_recognition_model.pkl')
    return model.predict(facearray)

In [11]:
def train_model():
    faces = []
    labels = []
    userlist = os.listdir('static/faces')
    for user in userlist:
        for imgname in os.listdir(f'static/faces/{user}'):
            img = cv2.imread(f'static/faces/{user}/{imgname}')
            resized_face = cv2.resize(img, (50, 50))
            faces.append(resized_face.ravel())
            labels.append(user)
    faces = np.array(faces)
    knn = KNeighborsClassifier(n_neighbors=5)
    knn.fit(faces,labels)
    joblib.dump(knn,'model/face_recognition_model.pkl')

In [12]:
def extract_attendance():
    df = pd.read_csv(f'Attendance/Attendance-{datetoday}.csv')
    names = df['Name']
    rolls = df['Roll']
    times = df['Time']
    l = len(df)
    return names,rolls,times,l

In [13]:
def add_attendance(name):
    username = name.split('_')[0]
    userid = name.split('_')[1]
    current_time = datetime.now().strftime("%H:%M:%S")
    
    df = pd.read_csv(f'Attendance/Attendance-{datetoday}.csv')
    if int(userid) not in list(df['Roll']):
        with open(f'Attendance/Attendance-{datetoday}.csv','a') as f:
            f.write(f'\n{username},{userid},{current_time}')

In [12]:
names,rolls,times,l = extract_attendance()

In [14]:
names,rolls,times,l

(0    Hao
 Name: Name, dtype: object,
 0    20040004
 Name: Roll, dtype: int64,
 0    22:08:41
 Name: Time, dtype: object,
 1)

In [11]:
def draw_smile(img, center, radius):
    axes = (radius, radius//2)
    color = (255,0,0)
    cv2.ellipse(img, center, axes, 0, 0, 180, color,thickness=2,lineType=cv2.LINE_AA)
    return

In [18]:
def detection(grayscale, img, face_detect, smile_detect):

    face = face_detect.detectMultiScale(grayscale, 1.3, 5)
    for (x,y,w,h) in face:

        ri_grayscale = grayscale[y:y+h, x:x+w]
        ri_color = img[y:y+h, x:x+w]
        
        
        cv2.rectangle(img,(x, y), (x+w, y+h), (255, 0, 20), 2)
        
        smile_lst = smile_detect.detectMultiScale(ri_grayscale, 1.7, 35)
        smiled = False
        
        for (x_smile, y_smile, w_smile, h_smile) in smile_lst:
            draw_smile(ri_color, (x_smile + w_smile // 2, y_smile + h_smile // 4), radius=w_smile // 3)
            smiled =  True

        if smiled:
            cv2.putText(face,"Smile",(10,30),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,0,20),2)
            # cv2.putText(img,"Smile",(100,100),cv2.FONT_HERSHEY_SIMPLEX,1.5,(0,0,255),3)
    return img

In [9]:
cap = cv2.VideoCapture(0)
ret = True
while ret:
    ret,frame = cap.read()
    if extract_faces(frame)!=():
        (x,y,w,h) = extract_faces(frame)[0]
        cv2.rectangle(frame,(x, y), (x+w, y+h), (255, 0, 20), 2)
        face = frame[y:y+h,x:x+w]
        cv2.putText(face,f'{x,y}',(10,20),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,0,20),2)

        (x,y,w,h) = smile_detector(frame)[0]
        face = frame[y:y+h,x:x+w]
        cv2.putText(face,"Smile",(10,30),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,0,20),2)
        
    cv2.imshow('Attendance',frame)
    if cv2.waitKey(1)==27:
        break
cap.release()
cv2.destroyAllWindows()

NameError: name 'extract_faces' is not defined

In [19]:
cap = cv2.VideoCapture(0)
ret = True

while ret:
    ret,frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame = detection(gray, frame,face_detector, smile_detector)

    cv2.imshow('Attendance',frame)
    if cv2.waitKey(1)==27:
        break
    
cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.6.0) :-1: error: (-5:Bad argument) in function 'putText'
> Overload resolution failed:
>  - img is not a numerical tuple
>  - Expected Ptr<cv::UMat> for argument 'img'


: 